In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from unidecode import unidecode
from sklearn.metrics import confusion_matrix,accuracy_score
import xgboost as xgb
# import polars as pl
import pandas as pd

In [52]:
def read_file(path):
  file = open(path)
  data = file.read()
  data_to_list_ = data.split("\n")
  data_to_list = (list(set(data_to_list_)))
  file.close()
  return data_to_list

In [99]:
tien_to_hau_to_1_word = read_file('./rule_files/tiento_hauto_1word.txt')
tien_to_hau_to_mutil_word = read_file('./rule_files/tiento_hauto_multi-word.txt')

ngoai_le_ca_nhan_mutil_word = read_file('./rule_files/ngoai_le_ca_nhan_mutil_word.txt')
ngoai_le_ca_nhan_one_word = read_file('./rule_files/ngoai_le_ca_nhan_one_word.txt')

special_name = read_file('./rule_files/special_person_name.txt')

non_person = read_file('./rule_files/non_person')

ngoai_le_one_word = ['codekhongsudung','code','inactive']
ngoai_le_mutil_word = ['huy code','huy do trung so','huy do trung thong tin','trung code','code huy']

In [100]:


# mutil word
def loc_khtc2(text):
    is_khtc = False
    for word in tien_to_hau_to_mutil_word:
        if word in text:
            is_khtc = True
            break
    return is_khtc



# once word
def loc_khtc1(text):
    words = text.split()
    is_khtc = False
    for word in words:
        if word in tien_to_hau_to_1_word:
            is_khtc = True
            break
    return is_khtc



# loc ten ngoai le
def loc_ngoai_le(text):
    is_ngoaile = False
    for word in non_person:
        if word in text:
            is_ngoaile = True
            break
    return is_ngoaile

def loc_ngoai_le_one_word(text):
    words = text.split()
    is_ngoai_le = False
    for word in words:
        if word in ngoai_le_one_word:
            is_ngoai_le = True
            break
    return is_ngoai_le
def loc_ngoai_le_mutil_word(text):
    is_ngoai_le = False
    for word in ngoai_le_mutil_word:
        if word in text:
            is_ngoai_le = True
            break
    return is_ngoai_le


# lọc ngoại lệ tên người
def loc_ngoai_le_ca_nhan_one_word(text):
    words = text.split()
    is_ngoai_le = False
    for word in words:
        if word in ngoai_le_ca_nhan_one_word:
            is_ngoai_le = True
            break
    return is_ngoai_le

def loc_ngoai_le_ca_nhan_mutil_word(text):
    is_ngoai_le = False
    for word in ngoai_le_ca_nhan_mutil_word:
        if word in text:
            print(word)
            is_ngoai_le = True
            break
    return is_ngoai_le
        

def has_special_char(s):
    for c in s:
        if not (c.isalpha() or c == ' '):
            return True
    return False


def has_one_word(s):
    if " " not in s:
        return True
    return False


def has_is_full_number(s):
    return s.isdigit()



def check_person_name(s):
    is_person_name = False
    for name in special_name:
        if s == name:
            is_person_name = True
    return is_person_name

In [101]:
path_data_train = 'data_generate/data_train_moredata5.csv'
model = xgb.XGBClassifier()

model.load_model("models/best_model.h5")

In [102]:
def preprocessing(real_x):

    data_train = pd.read_csv(path_data_train)

    x_train = data_train["name"]

    tfidf_vect_ngram_char = TfidfVectorizer(analyzer='char', max_features=30000, ngram_range=(2, 3))

    tfidf_vect_ngram_char.fit(x_train)    

    real_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(real_x)

    return real_tfidf_ngram_char




def predict_to_csv(id,real_x,file_path_export):

    youden_threshold = 0.2

    real_tfidf_ngram_char = preprocessing(real_x)

    real_predictions_proba = model.predict_proba(real_tfidf_ngram_char)

    real_predictions_proba_youden = (real_predictions_proba > youden_threshold).astype(int)

    real_predictions_proba_youden = real_predictions_proba_youden[:, 1]

    columns = ["id","name",  "ket qua du doan"]

    results = pd.DataFrame([id,real_x,  real_predictions_proba_youden])

    results= results.transpose()

    results.columns = columns

    results.to_csv(file_path_export)

    return file_path_export

In [103]:
MAKH = 'MAKH'
NAME = 'name'
# Run model to classify
def prediction(file_path):
    # real_data = pd.read_csv(file_path,sep=";")        
    # real_data= pd.read_csv(file_path)
    real_data= pd.read_excel(file_path)
    full_name =[]
    ids = []
    for i,id in enumerate(real_data[MAKH]):
        tenkh= str(real_data[NAME][i])
        #tenkh= real_data[NAME][i]
        if has_special_char(tenkh):
                real_data[NAME][i] = 'noname'
                full_name.append(tenkh)
                ids.append(id)
        else:
            try:
                full_name.append(unidecode(tenkh).lower())
                ids.append(id)
            except:
                real_data[NAME][i] = 'noname'
                full_name.append(tenkh)
                ids.append(id)
    real_data = pd.DataFrame([ids,full_name])
    real_data = real_data.transpose()
    real_data.columns=  [MAKH,NAME]
    real_x = real_data[NAME]
    file_name_export = file_path.split('/')[-1].split('.')[0]
    print(file_name_export)
    file_path_export = f"predict20M/{file_name_export}.csv"
    predict_to_csv(ids,real_x,file_path_export)
    return file_path_export


In [105]:
file_export = prediction('Book1.xlsx')

Book1


In [106]:
# Check rule-based
data =  pd.read_csv(file_export)
new_pred = []
for i in range(len(data)):
    new_pred.append(data['ket qua du doan'][i])
    ten_kh = str(data['name'][i])
    if data['ket qua du doan'][i]== 0:
        if loc_ngoai_le_ca_nhan_one_word(ten_kh) or loc_ngoai_le_ca_nhan_mutil_word(ten_kh):
            new_pred[i] = 1
        if check_person_name(ten_kh):
            new_pred[i] = 1
    if data['ket qua du doan'][i]==1:
        if loc_khtc2(ten_kh) or loc_khtc1(ten_kh):
            new_pred[i] = 0
    if loc_ngoai_le(ten_kh.lower()) or loc_ngoai_le_one_word(ten_kh.lower()) or loc_ngoai_le_mutil_word(ten_kh.lower()):
        new_pred[i] = 'ngoai le'
    if has_is_full_number(ten_kh) or 'gmail' in ten_kh:
        new_pred[i] = 'tap mo'
predict = np.array(new_pred)
data.insert(column='ket qua du doan new',value=predict,loc=4)
data.to_csv(file_export)